## Category/Subcategory Filtering

In [20]:
import pandas as pd
import collections
import numpy as np
pd.set_option('display.max_colwidth', -1)

In [86]:
#Some regex utility functions
import re

def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

In [25]:
#Loading data and dropping NAN rows
title_info = pd.read_csv('title_sub_cats.csv',index_col=False,names=['Title','category/subcategory'])
title_info = title_info.dropna()

#Splitting category and sub-category
title_info.replace({' >> ':''}, regex=True, inplace=True)
title_info['Category'], title_info['Subcategory'] = title_info['category/subcategory'].str.split(' >> ', 1).str
del title_info['category/subcategory']

#Shuffling data
title_info = title_info.iloc[np.random.permutation(len(title_info))]

#Resetting index
title_info = title_info.reset_index(drop=True)

title_info

,Title,Category,Subcategory
0,Summer House Oyster White 2-piece Entertainment Center,Living Room Furniture,Entertainment Centers
1,Somette Natural Reclaimed 3-drawer Media Console,Living Room Furniture,Entertainment Centers
2,TRIBECCA HOME Harrison Industrial Rustic Pipe Frame Media Stand with Two 40-inch Towers,Living Room Furniture,Entertainment Centers
3,Bermington Silver Wood and Fabric Dining Chairs (Set of 6),Dining Room & Bar Furniture,Dining Chairs
4,Republic Design House Archer Ivory Tufted Upholstered Headboard-Tufted Sofa Bench Collection,Bedroom Furniture,Headboards
5,Pasadena Gray Modular Sectional,Living Room Furniture,Sectional Sofas
6,Town and Country Transitional 24-inch Upholstered Barstool,Dining Room & Bar Furniture,Bar Stools
7,Zoe Cadet Blue Faux-Leather Oval Nightstand,Bedroom Furniture,Nightstands
8,Wyndham Collection Natalie 72-inch White UM Oval Sink Double Bathroom Vanity,Bathroom Furniture,Bathroom Vanities
9,"Linon Manhattanesque Backless Bar Stool, Black Vinyl Seat",Dining Room & Bar Furniture,Bar Stools


In [26]:
cat_vocab = dict(collections.Counter(title_info['Category']))
cat_df = pd.DataFrame(cat_vocab.items(),columns=['Category','Counts'])
cat_df

,Category,Counts
0,Home Office Furniture,1327
1,Bedroom Furniture,6210
2,Bathroom Furniture,1542
3,Living Room Furniture,7101
4,Dining Room & Bar Furniture,4940
5,Kids' Furniture,3491


In [123]:
subcat_vocab = dict(collections.Counter(title_info['Subcategory']))
subcat_df = pd.DataFrame(subcat_vocab.items(),columns=['Subcategory','Counts'])
subcat_df.sort_values('Counts', ascending=False)

,Subcategory,Counts
0,Ottomans,965
10,Bedroom Sets,945
44,Mattresses,940
18,Dining Sets,924
29,Bathroom Vanities,923
35,Dining Chairs,916
33,Bar Stools,913
34,Headboards,872
21,Dressers,856
13,Dining Tables,850


In [28]:
#Buliding Category-Subcategory Dictionary
cat_dict = {}
for c in cat_vocab:
    if c not in cat_dict.keys():
        cat_dict[c] = []

for i in range(len(title_info['Category'])):
    if title_info['Subcategory'][i].lower() not in cat_dict[title_info['Category'][i]]:
        cat_dict[title_info['Category'][i]].append(title_info['Subcategory'][i].lower())
    
catDict_df = pd.DataFrame(cat_dict.items(),columns=['Category', 'Subcategories'])
catDict_df

,Category,Subcategories
0,Living Room Furniture,"[entertainment centers, sectional sofas, benches, media/bookshelves, recliners, ottomans, living room sets, futons, sofas & loveseats, living room chairs, bean bag chairs]"
1,Dining Room & Bar Furniture,"[dining chairs, bar stools, pub sets, bar tables, buffets, dining sets, dining tables, bars]"
2,Home Office Furniture,"[office chairs, office storage & organization, tables, file cabinets, desks]"
3,Bedroom Furniture,"[headboards, nightstands, mattresses, bedroom sets, beds, dressers, bed frames, bedroom accents, bedroom mirrors, armoires, air beds]"
4,Bathroom Furniture,"[bathroom vanities, bathroom cabinets, bathroom shelving]"
5,Kids' Furniture,"[kids' nightstands, kids' dressers, kids' beds, cribs, bean & lounge bags, kids' chairs, kids' table & chair sets, kids' storage, crib mattresses, kids' desks, kids' bedroom sets, gliders & ottomans, changing tables, kids' tables, bassinets & cradles]"


In [31]:
sel_cat = {'Living Room Furniture': ['media/bookshelves','sofas & loveseats'],
           'Dining Room & Bar Furniture': ['dining sets','pub sets'],
           'Home Office Furniture':['office storage & organization','desks'],
           'Bedroom Furniture':['bedroom sets','mattresses'],
           'Bathroom Furniture':['bathroom cabinets','bathroom vanities']}

#Titles
tnames = ['titles']
tdata = pd.read_table('s3_per_sentence.train.features',index_col=False,names=tnames)
tdata.replace({'_EOT':'','_EOFE':''}, regex=True, inplace=True)
title_list = tdata['titles']
tdata

,titles
0,Titan TP-8300 Deluxe S-Track Deep Tissue Massage Chair
1,Titan TP-8300 Deluxe S-Track Deep Tissue Massage Chair Upholstery fill: Faux leather Care instructions: Clean with a damp cloth
2,Eco Stackable Connect Storage Cube with Door (made from sustainable non-toxic zBoard paperboard) Sustainably made from zBoard paperboard Material: Paperboard
3,Children's Chair (Set of 2) Material: Wood Furniture Pattern: Solid
4,Children's Chair (Set of 2)
5,Baxton Studio Cybele Solid Wood Dark Brown Twin over Twin Bunk Bed Bed Style: Bunk Bed Furniture Pattern: Solid
6,Baxton Studio Cybele Solid Wood Dark Brown Twin over Twin Bunk Bed Bed Style: Bunk Bed
7,Baxton Studio Cybele Solid Wood Dark Brown Twin over Twin Bunk Bed Finish options: Dark brown Color: Brown
8,Baxton Studio Cybele Solid Wood Dark Brown Twin over Twin Bunk Bed Furniture Pattern: Solid Size: Twin
9,"Child's Maple Bench Material: Maple Finish: Stained, Painted"


In [115]:
#Descriptions
f = open('s3_per_sentence.train.docs')
desc_list = f.read().splitlines()
desc_list = [strip_non_ascii(d) for d in desc_list]
ddata = pd.DataFrame(desc_list,columns=['descriptions'])
ddata.replace({'_EOS':''}, regex=True, inplace=True)
desc_list = ddata['descriptions']
ddata

,descriptions
0,Get a relaxing massage in the comfort of your own home with this Titan S-Track deep tissue massage chair.
1,This zero gravity massage chair has a faux leather covering that cleans up easily with a damp cloth.
2,"Made from zBoard recycled paperboard, this cube is eco-friendly as well as formaldehyde- and toxic-free."
3,"Made of 100 percent solid wood, these child size chairs are built strong for years of rough-and-tumble use in your home."
4,This set of two children's chairs is available in a variety of colors to suit your existing decor and bring a smile to your child's face.
5,"The Cybele bunk bed is a robust, modern style bunk, with clean, crisp lines and beautiful detailing, which is constructed from eco-friendly solid rubberwood."
6,This modern bunk bed takes its inspiration from the shaker style and features wide sturdy steps which are perfect for little feet.
7,"Finishing in dark brown color, the Cybele will match well with your kids' room decor."
8,A solid slatted base provides the perfect platform for standard size twin size mattress.
9,It is made of maple wood and can be stained in Harvest or painted white with colorful stencils.


In [116]:
title_dict = {}

for k,v in sel_cat.iteritems():
    for i in v:
        title_dict[(k,i)] = []
        count = 0
        for idx,ti in title_info.iterrows():
            if k.lower() == ti['Category'].lower() and i.lower() == ti['Subcategory'].lower():
                if len(title_dict[(k,i)]) <= 100:
                    title_dict[(k,i)].append(strip_non_ascii(ti['Title']))
                
titleDF = pd.DataFrame(title_dict.items(),columns=['Category,Subcategory', 'Titles'])
titleDF

Category,Subcategory  \
0  (Bathroom Furniture, bathroom cabinets)                  
1  (Home Office Furniture, office storage & organization)   
2  (Living Room Furniture, media/bookshelves)               
3  (Dining Room & Bar Furniture, dining sets)               
4  (Bedroom Furniture, mattresses)                          
5  (Living Room Furniture, sofas & loveseats)               
6  (Bedroom Furniture, bedroom sets)                        
7  (Home Office Furniture, desks)                           
8  (Bathroom Furniture, bathroom vanities)                  
9  (Dining Room & Bar Furniture, pub sets)                  

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [121]:
desc_dic = {}
for k,v in title_dict.iteritems():
    desc_dic[k] = []
    for i in v:
            for j in range(len(title_list)):
                #if findWholeWord(re.escape(i))(title_list[j]) != None:
                if i in title_list[j]:
                    desc_dic[k].append(desc_list[j])
    if len(desc_dic[k]) == 100:
        print 'Complete!'
descDF = pd.DataFrame(desc_dic.items(),columns=['Category,Subcategory', 'Descriptions'])
descDF

Category,Subcategory  \
0  (Bathroom Furniture, bathroom cabinets)                  
1  (Home Office Furniture, office storage & organization)   
2  (Living Room Furniture, media/bookshelves)               
3  (Dining Room & Bar Furniture, dining sets)               
4  (Bedroom Furniture, mattresses)                          
5  (Bedroom Furniture, bedroom sets)                        
6  (Home Office Furniture, desks)                           
7  (Bathroom Furniture, bathroom vanities)                  
8  (Living Room Furniture, sofas & loveseats)               
9  (Dining Room & Bar Furniture, pub sets)                  

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [124]:
descDF.to_csv('sentences_to_annotate.csv')